# Requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/shredder0812/deep-person-reid.git

# create environment
%cd deep-person-reid/
!pip create --name torchreid python=3.7
!pip activate torchreid

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
!pip install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 5089, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 5089 (delta 41), reused 29 (delta 13), pack-reused 5007
Receiving objects: 100% (5089/5089), 1.87 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (3532/3532), done.
/content/deep-person-reid
ERROR: unknown command "create"
ERROR: unknown command "activate"
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
/content/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
2024-05-10 09:20:58.989166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 09:20:58.989215: E external/local_xla/xla/stream_executor/cuda/cuda

In [ ]:
# create environment
%cd deep-person-reid/
!pip create --name torchreid python=3.7
!pip activate torchreid

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
!pip install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

In [ ]:
!mkdir /content/deep-person-reid/reid-data

In [ ]:
# make dataset dir ./deep-person-reid/reid-data/endocv
!cp -r /content/drive/MyDrive/endocv  /content/deep-person-reid/reid-data

In [ ]:
#!unzip /content/deep-person-reid/reid-data/endocv.zip -d /content/deep-person-reid/reid-data

# Main

In [ ]:
import torchreid

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="endocv",
    targets="endocv",
    # height=256,
    # width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded EndoCV
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    36 |      648 |         6
  query    |    36 |       83 |         6
  gallery  |    36 |      183 |         1
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded EndoCV
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    36 |      648 |         6
  query    |    36 |       83 |         6
  gallery  |    36 |      183 |         1
  ---------------------

/content/deep-person-reid/torchreid/data/datasets/image/endocv.py:34: UserWarning: The current data structure is deprecated. Please put data folders such as "bounding_box_train" under "Market-1501-v15.09.15".
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
model = torchreid.models.build_model(
    name="osnet_x0_25", #osnet_x0_25
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1rb8UN5ZzPKRc_xvtHlyDh-cSz88YX9hs
To: /root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth
100%|██████████| 2.97M/2.97M [00:00<00:00, 217MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)


In [ ]:
engine.run(
    save_dir="log/osnetx025endo",
    max_epoch=100,
    eval_freq=10,
    print_freq=10
    # if validate, enable 3 lines below
    # test_only=True,
    # visrank=True,
    # visrank_topk=10

)

=> Start training
epoch: [1/100][10/20]	time 0.265 (0.368)	data 0.005 (0.141)	eta 0:12:11	loss 1.2552 (1.3564)	acc 87.5000 (81.5625)	lr 0.000030
epoch: [1/100][20/20]	time 0.092 (0.262)	data 0.000 (0.073)	eta 0:08:39	loss 1.2145 (1.3180)	acc 90.6250 (83.4375)	lr 0.000030
epoch: [2/100][10/20]	time 0.373 (0.608)	data 0.001 (0.229)	eta 0:19:57	loss 1.4466 (1.2837)	acc 75.0000 (85.6250)	lr 0.000030
epoch: [2/100][20/20]	time 0.077 (0.467)	data 0.000 (0.150)	eta 0:15:16	loss 1.3087 (1.3079)	acc 78.1250 (83.5938)	lr 0.000030
epoch: [3/100][10/20]	time 0.329 (0.617)	data 0.005 (0.238)	eta 0:20:03	loss 1.2468 (1.2507)	acc 78.1250 (86.2500)	lr 0.000030
epoch: [3/100][20/20]	time 0.061 (0.394)	data 0.000 (0.137)	eta 0:12:44	loss 1.4012 (1.3086)	acc 84.3750 (82.6562)	lr 0.000030
epoch: [4/100][10/20]	time 0.389 (0.523)	data 0.014 (0.163)	eta 0:16:49	loss 1.3943 (1.3483)	acc 81.2500 (80.3125)	lr 0.000030
epoch: [4/100][20/20]	time 0.077 (0.427)	data 0.000 (0.113)	eta 0:13:40	loss 1.1799 (1.2971)	

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
%tensorboard --logdir /content/deep-person-reid/log/osnetx025endo

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-05-08 18:11:35.818599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 18:11:35.818674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 18:11:35.819903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 18:11:37.039222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow

In [ ]:
import tensorflow as tf
from tensorboard import notebook

notebook.start("--load_fast=false --logdir /content/deep-person-reid/log/osnetx025")

#

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-04-23 15:48:38.544925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 15:48:38.545002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 15:48:38.546258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 15:48:39.613973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 33, in <module>
    sys.exit(load_entry_point('tb-nightly', 'console_scripts', 'tensorboard

In [ ]:
%tensorboard --logdir /content/deep-person-reid/log/osnetx025

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-04-23 15:54:20.475363: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 15:54:20.475450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 15:54:20.477184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 15:54:22.867048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow

In [ ]:
import torch

model = torch.load('/content/drive/MyDrive/torchreid_model/osnet_x0_25_endocv.pt')


# Lưu mô hình vào tệp pt
torch.save(model, 'osnet_x0_25_endocv.pth')

In [ ]:
!python tools/visualize_actmap.py \
--root /content/deep-person-reid/reid-data \
-d endocv \
-m osnet_x0_25 \
--weights /content/deep-person-reid/osnet_x0_25_endocv.pt \
--save-dir log/visactmap_osnet_x0_25

2024-05-08 18:08:17.750741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 18:08:17.750803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 18:08:17.752466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 18:08:19.165956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ 

In [ ]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_x0_25',
    model_path='/content/deep-person-reid/osnet_x0_25_model_endo.pt',
    device='cuda'
)

image_list = [
    '/content/deep-person-reid/reid-data/endocv/query/0001_c1s2_000226_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0002_c2s1_000205_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0003_c3s1_000125_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0004_c4s1_000137_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0005_c5s2_000039_01.jpg',
    '/content/deep-person-reid/reid-data/endocv/query/0006_c6s2_000014_01.jpg'
]

features = extractor(image_list)
print(features.shape) # output (5, 512)

Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "/content/deep-person-reid/log/osnetx025/model/model.pth.tar-50"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
torch.Size([6, 512])


In [ ]:
print(features)

tensor([[1.2108, 0.0000, 0.5007,  ..., 0.0000, 0.3413, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.2481, 0.2577, 1.2484],
        [3.1901, 0.0000, 0.9175,  ..., 0.0000, 0.2050, 0.0000],
        [0.0000, 3.2798, 0.0000,  ..., 0.0000, 0.7325, 2.1805],
        [0.0000, 0.7419, 0.0000,  ..., 1.4197, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.7335,  ..., 0.0000, 0.5269, 0.0000]],
       device='cuda:0')


In [ ]:
!cp -r /content/deep-person-reid/log/osnetx025_2/visrank_endocv /content/drive/MyDrive/logs